# Christopher Prince [cmp670@nyu.edu]
## MATH.2012 HW 2

### _Preliminaries_

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import pylab as pl
%matplotlib inline

## Source files and reproducibility

The program has been packaged with a make file and a bash script to run the experiment on a given machine. The process is:

* Clone the github repo cmprince/hpc17 (files are in the hw2 directory).
* Run `make`.
* Run `runlaplace.sh` (if necessary change file permissions). This runs both Gauss-Seidel and Jacobi iterations for $N \in \{50, 100, 1000\}$, repeating 5 times, and writes the results of each run to a file.

----

## Architectures

The tests were run on 9 different machines: my personal laptop, a server at NYU-CUSP, and two servers at CIMS. The specifications for the CUSP server was obtained from datahub.cusp.nyu.edu, and for the CIMS server from cims.nyu.edu.

| Machine  | Owner | Processor        | No. Processors | No. Cores/Proc. | Freq. (GHz) | Memory (GB) | Operating System |
|----------|-------|------------------|----------------|-----------------|-------------|-------------|----|
| brawler  | CIMS  | AMD Opteron      | 2              | 2               | 2.6         | 8           | RHEL Server 6.8
| compute  | CUSP  | Xeon E5-4640     | 4              | 8               | 2.40        | 1024        | Oracle Linux Server 6.4
| crunchy4 | CIMS  | AMD Opteron 6136 | 4              | 8               | 2.4         | 128         | RHEL Server 6.8
| laptop   | Me    | Xeon E3-1505M v5 | 1              | 4               | 2.80        | 64          | Ubuntu 16.04 LTS

## Results and observations

### Timing results
The results files are space-delimited with the following columns:

In [3]:
names=['N','run','usec']

Uncomment the cell below to retrieve the sample data used in this notebook:

In [4]:
#!wget -r -l1 -np -nd -R index.html?*,html -e robots=off https://www.cims.nyu.edu/~cmp670/hw2_output/

Read the files in as dataframes and merge the timing columns, starting with Gauss-Seidel:

In [5]:
gb = pd.read_csv("jacobi2D-omp_laptop", header=None, delimiter=" ", names=names).drop("usec", axis=1)

In [6]:
for f in sorted(os.listdir('.')):
    if f[:13]=="jacobi2D-omp_":
        g = pd.read_csv(f, header=None, delimiter=" ",names=names[:-1] + [f])
        gb = gb.merge(g, on=names[:-1])

In [7]:
gb

,N,run,jacobi2D-omp_brawler,jacobi2D-omp_brawler_no_l2norm_parallel,jacobi2D-omp_crunchy4,jacobi2D-omp_laptop,jacobi2D-omp_mauler_no_l2norm_parallel
0,50,1,64098,122950,3869467,79251,122773
1,50,2,62954,324677,3903125,25442,123221
2,50,3,62743,122595,3926278,24723,171922
3,50,4,64429,122490,3864763,21033,125596
4,50,5,62655,122655,3871138,20687,124616
5,100,1,225179,492691,3942070,73839,488551
6,100,2,224954,685548,3914154,72827,1076625
7,100,3,230868,1062484,405486,76498,765194
8,100,4,230655,596687,3895513,72453,488826
9,100,5,224027,579145,3924761,78413,483022


There are a few runs where the timing is much greater than the others; instead of taking a mean value over the runs I calculate the median run time here.

In [8]:
g = gb.groupby(names[0])
g.agg(np.median).drop('run', axis=1)

,jacobi2D-omp_brawler,jacobi2D-omp_brawler_no_l2norm_parallel,jacobi2D-omp_crunchy4,jacobi2D-omp_laptop,jacobi2D-omp_mauler_no_l2norm_parallel
N,,,,,
50,62954,122655,3871138,24723,124616
100,225179,596687,3914154,73839,488826
1000,21845399,47086142,6944077,6500057,47643289


Repeating for Jacobi:

In [9]:
gb = pd.read_csv("gs2D-omp_laptop", header=None, delimiter=" ", names=names).drop("usec", axis=1)

In [10]:
for f in sorted(os.listdir('.')):
    if f[:9]=="gs2D-omp_":
        g = pd.read_csv(f, header=None, delimiter=" ", names=names[:-1] + [f])
        gb = gb.merge(g, on=names[:-1])

In [11]:
gb

,N,run,gs2D-omp_brawler,gs2D-omp_crunchy4,gs2D-omp_laptop,gs2D-omp_mauler,gs2D-omp_mauler_no_l2norm_parallel
0,50,1,124188,3893753,85588,134271,134271
1,50,1,125567,3893753,85588,134271,134271
2,50,2,125482,3838401,85986,126126,126126
3,50,2,125139,3838401,85986,126126,126126
4,50,3,124588,3890386,80742,179788,179788
5,50,3,125385,3890386,80742,179788,179788
6,50,4,124582,3876093,84353,125224,125224
7,50,4,125531,3876093,84353,125224,125224
8,50,5,125483,3887672,83798,124929,124929
9,50,5,124839,3887672,83798,124929,124929


In [12]:
g = gb.groupby(names[0])
g.agg(np.median).drop('run', axis=1)

,gs2D-omp_brawler,gs2D-omp_crunchy4,gs2D-omp_laptop,gs2D-omp_mauler,gs2D-omp_mauler_no_l2norm_parallel
N,,,,,
50,125262,3887672,84353,126126,126126
100,483575,3886792,311661,487323,487323
1000,46464023,32292971,25087905,46865096,46865096
